The only input para is elf_file name.

You can tune top_counts to show more items.

The output is total size of all symbol types, and detail of top top_counts of each type.

In [1]:
import os
import re
import pandas as pd
import numpy as np

# 1 Pre-processing
1. Dump elf file symbols by readelf -s xxx
2. Read 'size, address, type, name' of each line
3. Append all raw data to elf_lists

In [2]:
elf_file = 'iot_ap.elf'
elf_summary = elf_file.split('.')[0]
elf_lists = []
top_counts = 20

if not os.path.exists(elf_summary):
    os.mkdir(elf_summary)

#elf_summary_object = open(elf_summary, 'wb')

tmp = os.popen('readelf -s %s' % elf_file).readlines()
elf_symbol_fmt = ' *(?P<num>[0-9]*): (?P<value>[0-9abcdef]*) *(?P<size>[0-9]*).*'
for line in tmp:
    m = re.match(elf_symbol_fmt, line)
    if not m:
        continue
    #num = m.group('num')
    elf_line_list = re.split(r'\s+', line)

    if elf_line_list[3][0:2] == '0x':
        elf_line_list[3] = int(elf_line_list[3][2:], 16)
    #                     size             address            tyoe             name
    elf_lists.append([elf_line_list[3], elf_line_list[2], elf_line_list[4], elf_line_list[8]])
#elf_summary_object.writelines(tmp)
#elf_summary_object.close()

# 2 Processing
1. Sort all data by size values
2. Sort all FUNC data by its size values
3. Sort all OBJECT data by its size values

In [3]:
elf_data = pd.DataFrame(np.asarray(elf_lists), columns=['size', 'address', 'type', 'name'])
elf_data['size'] = elf_data['size'].astype(int)

#elf_data.sort_values(by=['size', 'type'], ascending=[False, False]).head(top_counts).to_csv(elf_summary, mode='w')
top_all_head = pd.DataFrame(elf_data.sort_values(by=['size', 'type'], ascending=[False, False]))

top_func_head = elf_data[elf_data['type'] == 'FUNC'].sort_values(by=['size'], ascending=False)
top_object_head = elf_data[elf_data['type'] == 'OBJECT'].sort_values(by=['size'], ascending=False)

# 3 Post processing
1. Give a summary of each type
2. Show top top_counts of FUNC and OBJECT

In [4]:
elf_types = elf_data['type'].unique()
totalsize_of_types = 0

print '\nSize of %s:' % (elf_types)
for i in elf_types:
    size_of_type = np.asarray(elf_data[elf_data['type'] == i].sort_values(by=['size'], ascending=False)['size']).sum()
    print i, ':', size_of_type, 'Bytes'
    totalsize_of_types += size_of_type
print 'Total : ', totalsize_of_types/1024, 'KB'


Size of ['NOTYPE' 'SECTION' 'FILE' 'FUNC' 'OBJECT']:
NOTYPE : 0 Bytes
SECTION : 0 Bytes
FILE : 0 Bytes
FUNC : 141478 Bytes
OBJECT : 77770 Bytes
Total :  214 KB


In [5]:
top_all_head.to_csv('%s/top_all.csv' % elf_summary)
print '\nThe top %d of %s:' % (top_counts, elf_file)
top_all_head.head(top_counts)


The top 20 of iot_ap.elf:


,size,address,type,name
5312,20648,00018960,OBJECT,_mpool_buf__heap_mem_pool
3584,5200,00014d4c,OBJECT,g_instLightList
5091,4096,000175c4,OBJECT,_sys_k_event_logger_buffe
3586,3120,0001619c,OBJECT,g_instTempList
653,2048,0001e060,OBJECT,ipv6_udp_stack
655,2048,0001f060,OBJECT,ipv4_udp_stack
704,2048,00012ccc,OBJECT,thread_stack.5013
4859,2048,00022228,OBJECT,_interrupt_stack
5405,2048,0001e860,OBJECT,thread_stack
3028,2000,000207b8,OBJECT,stack


In [6]:
top_object_head.to_csv('%s/top_object.csv' % elf_summary)
print '\nThe top %d of %s:' % (top_counts, 'OBJECT')
top_object_head.head(top_counts)


The top 20 of OBJECT:


,size,address,type,name
5312,20648,00018960,OBJECT,_mpool_buf__heap_mem_pool
3584,5200,00014d4c,OBJECT,g_instLightList
5091,4096,000175c4,OBJECT,_sys_k_event_logger_buffe
3586,3120,0001619c,OBJECT,g_instTempList
4859,2048,00022228,OBJECT,_interrupt_stack
704,2048,00012ccc,OBJECT,thread_stack.5013
5405,2048,0001e860,OBJECT,thread_stack
655,2048,0001f060,OBJECT,ipv4_udp_stack
653,2048,0001e060,OBJECT,ipv6_udp_stack
3028,2000,000207b8,OBJECT,stack


In [7]:
top_func_head.to_csv('%s/top_func.csv' % elf_summary)
print '\nThe top %d of %s:' % (top_counts, 'FUNC')
top_func_head.head(top_counts)


The top 20 of FUNC:


,size,address,type,name
4585,1750,0ffb8b99,FUNC,_prf
4827,1704,0ffb03e9,FUNC,__aeabi_dsub
4732,1624,0ffaf055,FUNC,__aeabi_dadd
4584,1580,0ffaf6ad,FUNC,__aeabi_ddiv
4843,1382,0ffcce5f,FUNC,observe_step
2409,1348,0ffbf625,FUNC,handle_ra_input
4603,1268,0ffafef5,FUNC,__aeabi_dmul
4397,1180,0ffb2621,FUNC,ramdump_trans_to_pc
5013,1168,0ffca115,FUNC,cis_sync_entry
4816,1116,0ffcc111,FUNC,management_handleRequest
